In [34]:
from Bio.Seq import Seq
import pandas as pd
import os
import math

In [35]:
"""
Input: a txt file containing gRNA sequences. The file can either be one column of gRNA sequences only, or the output file from 1_SNP_gRNA

Output: a txt file containing primers that assemble gRNAs into the CREEPY vector
"""

'\nInput: a txt file containing gRNA sequences. The file can either be one column of gRNA sequences only, or the output file from 1_SNP_gRNA\n\nOutput: a txt file containing primers that assemble gRNAs into the CREEPY vector\n'

In [36]:
#Input filename
gRNA_FILENAME = 'grna_pam_strand_pXW487.txt'

#Output filename
creepy_primer_file = ('creepy_primers_new')

desktop = os.path.expanduser("~/Desktop")
gRNA_file_path = desktop + '/' + gRNA_FILENAME
creepy_primer_path = desktop + '/' + creepy_primer_file


In [37]:
# gRNA_df = pd.read_csv(gRNA_file_path, sep='\t', header=None)
# gRNA_df
def read_data(file_path):
    try:
        df = pd.read_csv(file_path, sep=None)
        if "SNP" in df.columns and "gRNA" in df.columns:
            df["SNP"] = df["SNP"].astype(str)
            df["gRNA"] = df["gRNA"].str.replace(" ", "")
            df["gRNA"] = df["gRNA"].apply(lambda seq_str: Seq(seq_str.upper()))
            return df
    except pd.errors.EmptyDataError:
        pass

    try:
        df = pd.read_csv(file_path, header=None, names=["gRNA"])
        df['SNP'] = range(1, df.shape[0]+1)
        df['SNP'] = df['SNP'].astype(str)
        df["gRNA"] = df["gRNA"].str.replace(" ", "")
        df["gRNA"] = df["gRNA"].apply(lambda seq_str: Seq(seq_str.upper()))
        return df
    except FileNotFoundError:
        pass

    return None

In [38]:
gRNA_df = read_data(gRNA_file_path)
gRNA_df

C:\Users\wxh02\AppData\Local\Temp\ipykernel_23532\2261349770.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep=None)


,SNP,gRNA,PAM,PAM_pos,strand
0,rs10774035,"(C, C, C, C, T, T, G, C, A, A, C, A, T, C, T, ...",CGG,82431,-
1,rs10774036,"(C, A, A, T, T, T, T, T, G, G, T, G, A, G, C, ...",TGG,100699,-
2,rs10744560,"(A, G, C, A, C, G, A, T, G, A, G, T, G, A, G, ...",AGG,100847,-
3,rs12311439,"(A, C, T, C, T, G, C, T, G, T, G, C, C, G, C, ...",TGG,108536,+
4,rs1024582,"(G, T, A, T, T, A, T, T, T, G, G, T, T, G, T, ...",GGG,116001,+
5,rs4298967,"(A, G, G, C, T, G, A, A, C, G, G, A, G, C, T, ...",GGG,121942,+


In [39]:
def creepy_primer(df):
    """
    Design CREEPY primers based on a list of gRNAs
    For one guide RNA, return two oligos that generate sticky ends after annealing.
    For two guide RNAs, return two primers, with each primer incorperating one gRNA.
    For three and more guide RNAs, return a set of primers:
        The first and last primers introduce the first and last gRNAs, respectively.
        The rest of the primers introduce the middle gRNAs that are each split into two halves,
        with the first half in the reverse primer and the second half in the fwd primer.
    :param df:
    :return: a dictionary of primers, and a dictionary of sticky end sequences
    """
    sticky_end_left = Seq('GACT')    #left sticky end of pXW467/468/472
    sticky_end_right = Seq('AAAC')   #right sticky end of pXW67/468/472
    template_fwd = Seq('GTTTTAGAGCTAGAAATAGCAAGTTA') #Fwd primer to amplify scaffold RNA
    template_rev = Seq('TGCGCAAGCCCGGAATCGAACCGGG')  #Rev primer to amplify tRNA_Gly
    adaptor_left = Seq('aCGTCTCagacttt') #left adaptor for Esp3I to generate a GACT sticky end followed by tt
    adaptor_right = Seq('aCGTCTCcaaac')  #Right adaptor for Esp3I to generate a AAAC sticky end
    adaptor3 = Seq('aCGTCTCc')   #universal adaptor for Esp3I, will generate a sticky end that matches the 4 following nt
    overhang_set = {"left": sticky_end_left, "right": sticky_end_right}
    
    if df.shape[0] == 1:
        # For one gRNA only
        primer1_name, primer1 = df['SNP'].iloc[0] + '_F', sticky_end_left + 'tt' + df['gRNA'][0]
        primer2_name, primer2 = df['SNP'].iloc[0] + '_R', sticky_end_right + df['gRNA'][0].reverse_complement() + 'aa'
        p_set = {primer1_name: primer1, primer2_name: primer2}

    elif df.shape[0] == 2:
        # For two gRNAs
        primer1_name, primer1 = df['SNP'].iloc[0] + '_F', adaptor_left + df['gRNA'][0] + template_fwd
        primer2_name, primer2 = df['SNP'].iloc[1] + '_R', adaptor_right + df['gRNA'][1].reverse_complement() + template_rev
        p_set = {primer1_name: primer1, primer2_name: primer2}

    else:
        # For three and more gRNAs (1st and last are introduced in one primer, all others are split into two primers and joined by golden gate
        overhang_start_list = []
        p_set = {}
        primer1_name = df['SNP'].iloc[0] + '_F'
        p_set[primer1_name] = adaptor_left + df['gRNA'][0] + template_fwd
        overhang_list = list(overhang_set.values())

        for n in range(1,len(df['gRNA'])-1):
            gRNA_seq = df['gRNA'][n]#.upper()
            overhang_start = 8
            overhang = gRNA_seq[overhang_start:overhang_start + 4]
            i=1
            while overhang.reverse_complement() == overhang or overhang in overhang_list or overhang.reverse_complement() in overhang_list:
                overhang_start = 8 + math.ceil(i/2) * (-1) ** i
                overhang = gRNA_seq[overhang_start:overhang_start+4]
                i = i + 1
                
            overhang_set[df['SNP'][n]] = f"{str(overhang)} start_pos: ({overhang_start+1})"
            overhang_list.append(overhang)
            p_r_name = df['SNP'].iloc[n] + "_aR"
            p_f_name = df['SNP'].iloc[n] + "_bF"
            p_set[p_r_name] = adaptor3 + gRNA_seq[:overhang_start+4].reverse_complement() + template_rev
            p_set[p_f_name] = adaptor3 + gRNA_seq[overhang_start:] + template_fwd

        primer_last_name = df['SNP'].iloc[-1] + "_R"
        p_set[primer_last_name] = adaptor_right + df['gRNA'].iloc[-1].reverse_complement() + template_rev
        print(overhang_list)
    return  p_set, overhang_set


In [40]:
primer_set, sticky_end_set = creepy_primer(gRNA_df)

with open(creepy_primer_path, "w") as file:
    for key, value in primer_set.items():
        file.write(f"{key}:\t{value}\n")
    file.write('\n\nSticky ends:\n\n')
    for key, value in sticky_end_set.items():
        file.write(f"{key}:\t{value}\n")    


[Seq('GACT'), Seq('AAAC'), Seq('GGTG'), Seq('GAGT'), Seq('GTGC'), Seq('TGGT')]
